In [ ]:
import path
import libs.formulaSym2 as formulaSym
import libs.dReal as dReal
from control import *
from z3 import *


In [ ]:
# Constants
Kp = Real('Kp')
zeta =  Real('zeta')
omega_n = Real('omega_n')

f1 = zeta * zeta == 0.043 / Kp - 0.065
f2 = omega_n * omega_n == 11.29 * Kp
f3 = zeta >= 0

# Setup
f = formulaSym.Formula(zeta, omega_n)
s = Solver()

# Safety
safety = f.overshoot == 0

# PO generation
s.add(f1, f2, f3)
PO = Implies(And(f.EMp_1(), f.EMp_2()), safety)
s.add(Not(PO))

Let us use dReal to solve this proof obligation via SMT2 interface:

In [ ]:
# solving in dReal
dR = dReal.dReal()
smt2 = dR.getSMT2(s.sexpr())
dR.save(smt2)
model = dR.exe()

As we can see, the *sat* suggests a counter-example is found. In this case, dReal find a *zeta* within the given range, it causes overshoot = 1.198..., which violates the safety requirement: overshoot = 0.

Let us try the same proof obligation in Z3:

In [ ]:
# Symbolic Static Checking in Z3 SMT solver
print(f"static checking result =  {s.check()}")
if s.check() == sat:
    print(f"static checking model =  {s.model()}")

As we can see, Z3 return *unknown* in this case. 

However, not all SMT2 syntax are supported by dReal (checked on Aug 28, 2021), e.g. Array, uninterpreted functions. Thus, it would be difficult for us to encode our static checker entirely into dReal.

Therefore, it is advisable to perform interactive incremental checking for complex hybrid designs.
